In [1]:
import pandas as pd
import numpy as np

# Step 1: Define the columns
months = pd.date_range(start='2023-04-01', end='2024-02-01', freq='MS')  # Monthly start dates
ledger_values = ['Ledger_1', 'Ledger_2', 'Ledger_3', 'Ledger_4']  # 4 unique ledgers
customer_groups = [f'Customer_Group_{i+1}' for i in range(250)]  # 250 unique customer groups

# Step 2: Create the dataset
rows = 1000
np.random.seed(42)  # For reproducibility

data = {
    'Month': np.random.choice(months, rows),  # Randomly select months
    'Ledger': np.random.choice(ledger_values, rows),  # Randomly select ledgers
    'Customer Group': np.random.choice(customer_groups, rows),  # Randomly select customer groups
    'INR.2': np.random.rand(rows) * 10000  # Random INR.2 values as target
}

# Step 3: Create DataFrame
df = pd.DataFrame(data)




In [2]:
df.head()

,Month,Ledger,Customer Group,INR.2
0,2023-10-01,Ledger_3,Customer_Group_72,6809.748272
1,2023-07-01,Ledger_4,Customer_Group_60,1896.948482
2,2024-02-01,Ledger_2,Customer_Group_174,6189.481930
3,2023-11-01,Ledger_4,Customer_Group_217,3235.003949
4,2023-08-01,Ledger_3,Customer_Group_145,8343.033660


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Month           1000 non-null   datetime64[ns]
 1   Ledger          1000 non-null   object        
 2   Customer Group  1000 non-null   object        
 3   INR.2           1000 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 31.4+ KB


In [4]:
df['Month'].unique()

<DatetimeArray>
['2023-10-01 00:00:00', '2023-07-01 00:00:00', '2024-02-01 00:00:00',
 '2023-11-01 00:00:00', '2023-08-01 00:00:00', '2024-01-01 00:00:00',
 '2023-06-01 00:00:00', '2023-09-01 00:00:00', '2023-05-01 00:00:00',
 '2023-04-01 00:00:00', '2023-12-01 00:00:00']
Length: 11, dtype: datetime64[ns]

In [5]:
df = df.sort_values(by=['Month', 'Ledger', 'Customer Group'])

In [6]:
df.head()

,Month,Ledger,Customer Group,INR.2
431,2023-04-01,Ledger_1,Customer_Group_1,2155.298688
947,2023-04-01,Ledger_1,Customer_Group_137,3976.475796
812,2023-04-01,Ledger_1,Customer_Group_142,2422.312178
773,2023-04-01,Ledger_1,Customer_Group_144,6325.441100
211,2023-04-01,Ledger_1,Customer_Group_148,1782.524669


In [7]:
aggregated_df = df.groupby(['Month', 'Ledger', 'Customer Group']).agg({'INR.2': 'mean'}).reset_index()


In [8]:
aggregated_df.head()

,Month,Ledger,Customer Group,INR.2
0,2023-04-01,Ledger_1,Customer_Group_1,2155.298688
1,2023-04-01,Ledger_1,Customer_Group_137,3976.475796
2,2023-04-01,Ledger_1,Customer_Group_142,2422.312178
3,2023-04-01,Ledger_1,Customer_Group_144,6325.441100
4,2023-04-01,Ledger_1,Customer_Group_148,1782.524669


In [32]:
aggregated_df.tail()

,Month,Ledger,Customer Group,INR.2
954,2024-02-01,Ledger_4,Customer_Group_43,5791.752554
955,2024-02-01,Ledger_4,Customer_Group_49,2693.312983
956,2024-02-01,Ledger_4,Customer_Group_80,1828.072367
957,2024-02-01,Ledger_4,Customer_Group_92,441.619362
958,2024-02-01,Ledger_4,Customer_Group_97,950.116990


In [33]:
def forecast_next_3_months(group_df, current_date):
    group_df = group_df.sort_values('Month')
    last_three_months = group_df[group_df['Month'] < current_date].tail(3)  # Get last 3 months' data
    forecast_value = last_three_months['INR.2'].mean()  # Simple average (can replace with more advanced methods)

    # Forecast for next 3 months (January, February, March if current_date is January 1)
    forecast_dates = pd.date_range(start=current_date, periods=3, freq='MS')  # Next 3 months
    forecast_df = pd.DataFrame({
        'Month': forecast_dates,
        'Ledger': group_df['Ledger'].iloc[0],
        'Customer Group': group_df['Customer Group'].iloc[0],
        'INR.2 Forecast': forecast_value
    })

    return forecast_df




In [34]:

# Step 2: Apply the forecast function to each Ledger and Customer Group combination
forecast_results = pd.DataFrame()
current_date = pd.to_datetime('2024-03-01')

for (ledger, customer_group), group_df in aggregated_df.groupby(['Ledger', 'Customer Group']):
    forecast_df = forecast_next_3_months(group_df, current_date)
    forecast_results = pd.concat([forecast_results, forecast_df], ignore_index=True)

In [35]:
# Step 3: Display the forecasted values for next 3 months (Jan, Feb, Mar 2024)
forecast_results

,Month,Ledger,Customer Group,INR.2 Forecast
0,2024-03-01,Ledger_1,Customer_Group_1,2155.298688
1,2024-04-01,Ledger_1,Customer_Group_1,2155.298688
2,2024-05-01,Ledger_1,Customer_Group_1,2155.298688
3,2024-03-01,Ledger_1,Customer_Group_101,6418.599589
4,2024-04-01,Ledger_1,Customer_Group_101,6418.599589
...,...,...,...,...
1915,2024-04-01,Ledger_4,Customer_Group_97,950.116990
1916,2024-05-01,Ledger_4,Customer_Group_97,950.116990
1917,2024-03-01,Ledger_4,Customer_Group_99,445.949084
1918,2024-04-01,Ledger_4,Customer_Group_99,445.949084


In [36]:
forecast_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1920 entries, 0 to 1919
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Month           1920 non-null   datetime64[ns]
 1   Ledger          1920 non-null   object        
 2   Customer Group  1920 non-null   object        
 3   INR.2 Forecast  1920 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 60.1+ KB


In [37]:
forecast_results.isnull().sum()

,0
Month,0
Ledger,0
Customer Group,0
INR.2 Forecast,0


# Arima forecasting

In [14]:
!pip install statsmodels


In [38]:
def forecast_arima(group_df, current_date):
    group_df = group_df.sort_values('Month')
    past_data = group_df[group_df['Month'] < current_date]  # Past data

    # Ensure there is enough data to train the model (minimum 3 data points for ARIMA)
    if len(past_data) < 3:
        return None

    # Fit the ARIMA model (order can be tuned)
    model = ARIMA(past_data['INR.2'], order=(1, 1, 1))
    model_fit = model.fit()

    # Forecast for the next 3 months (Jan, Feb, Mar if current_date is Jan 1)
    forecast_dates = pd.date_range(start=current_date, periods=3, freq='MS')
    forecast_values = model_fit.forecast(steps=3)  # Predict next 3 months

    forecast_df = pd.DataFrame({
        'Month': forecast_dates,
        'Ledger': group_df['Ledger'].iloc[0],
        'Customer Group': group_df['Customer Group'].iloc[0],
        'INR.2 Forecast': forecast_values
    })

    return forecast_df

In [39]:
from statsmodels.tsa.arima.model import ARIMA

In [40]:
forecast_results = pd.DataFrame()
current_date = pd.to_datetime('2024-03-01')

for (ledger, customer_group), group_df in aggregated_df.groupby(['Ledger', 'Customer Group']):
    forecast_df = forecast_arima(group_df, current_date)

    # Only append if forecast_df is not None
    if forecast_df is not None:
        forecast_results = pd.concat([forecast_results, forecast_df], ignore_index=True)


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No suppor

In [41]:
print(forecast_results)


         Month    Ledger      Customer Group  INR.2 Forecast
0   2024-03-01  Ledger_1  Customer_Group_117     3108.048756
1   2024-04-01  Ledger_1  Customer_Group_117     3524.289764
2   2024-05-01  Ledger_1  Customer_Group_117     3400.351056
3   2024-03-01  Ledger_1   Customer_Group_12     6505.064369
4   2024-04-01  Ledger_1   Customer_Group_12     6488.356805
..         ...       ...                 ...             ...
187 2024-04-01  Ledger_4   Customer_Group_49     2693.564862
188 2024-05-01  Ledger_4   Customer_Group_49     3415.334907
189 2024-03-01  Ledger_4   Customer_Group_60     2519.651304
190 2024-04-01  Ledger_4   Customer_Group_60     3920.050295
191 2024-05-01  Ledger_4   Customer_Group_60     3476.937111

[192 rows x 4 columns]


#XGboost and lstm

In [42]:
def create_lag_features(group_df, lags=3):
    group_df = group_df.sort_values('Month')

    for lag in range(1, lags+1):
        group_df[f'INR.2_lag_{lag}'] = group_df['INR.2'].shift(lag)

    # Drop rows with NaN values (because they don't have sufficient lag data)
    group_df = group_df.dropna().reset_index(drop=True)

    return group_df

In [43]:
prepared_data = pd.DataFrame()

for (ledger, customer_group), group_df in aggregated_df.groupby(['Ledger', 'Customer Group']):
    lagged_df = create_lag_features(group_df)
    prepared_data = pd.concat([prepared_data, lagged_df], ignore_index=True)

In [44]:
prepared_data.head()


,Month,Ledger,Customer Group,INR.2,INR.2_lag_1,INR.2_lag_2,INR.2_lag_3
0,2024-01-01,Ledger_1,Customer_Group_198,5201.905233,4452.461123,3790.573211,6874.063641
1,2023-07-01,Ledger_1,Customer_Group_8,8192.551049,646.166798,4764.833838,9880.655667
2,2023-08-01,Ledger_1,Customer_Group_8,3559.733738,8192.551049,646.166798,4764.833838
3,2023-10-01,Ledger_1,Customer_Group_93,6292.988060,5202.965769,2852.903384,4681.541413
4,2023-07-01,Ledger_2,Customer_Group_141,7962.405956,5020.502135,2184.431829,3993.687484


In [45]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Step 2: Prepare training and test sets
X = prepared_data[['INR.2_lag_1', 'INR.2_lag_2', 'INR.2_lag_3']]  # Lag features
y = prepared_data['INR.2']  # Target column

# Split data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
model.fit(X_train, y_train)

# Step 4: Make predictions
y_pred = model.predict(X_test)

# Step 5: Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Root Mean Squared Error: {rmse}")


Root Mean Squared Error: 4756.708345786184


In [46]:
# Example: Forecast for next month using the last known 3 months of INR.2 values
last_known_values = prepared_data[['INR.2_lag_1', 'INR.2_lag_2', 'INR.2_lag_3']].iloc[-1].values.reshape(1, -1)

# Predict next month's INR.2 value
next_month_forecast = model.predict(last_known_values)
print(f"Forecasted INR.2 value for next month: {next_month_forecast[0]}")


Forecasted INR.2 value for next month: 6945.21337890625


In [26]:
#LSTM

In [47]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Step 1: Prepare data for LSTM (reshape input into [samples, time steps, features])
X = prepared_data[['INR.2_lag_1', 'INR.2_lag_2', 'INR.2_lag_3']].values
y = prepared_data['INR.2'].values

# Reshape X into [samples, time steps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

# Step 2: Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Build LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Step 4: Train the LSTM model
model.fit(X_train, y_train, epochs=50, verbose=1)

# Step 5: Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"LSTM Root Mean Squared Error: {rmse}")

# Forecast for the next month
last_known_values = X[-1].reshape(1, X.shape[1], 1)
next_month_forecast = model.predict(last_known_values)
print(f"LSTM forecast for next month: {next_month_forecast[0][0]}")


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 27752022.0000
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 26913552.0000
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 26688016.0000
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 26440566.0000
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 26170448.0000
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 25800086.0000
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 25470058.0000
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 25186408.0000
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 24865486.0000
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 24554606.0000
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 24188238.0000
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 23580976.0000
Epoch 13/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 23466110.0000
Epoch 14/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 23151224.0000
Epoch 15/50
1/

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
LSTM forecast for next month: 2034.6480712890625


In [48]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# For XGBoost or LSTM predictions, evaluate model performance

def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"R-squared (R²): {r2}")

# Assuming y_test and y_pred are the true and predicted values
evaluate_model(y_test, y_pred)


Mean Squared Error (MSE): 3651976.965833977
Root Mean Squared Error (RMSE): 1911.0146430192463
Mean Absolute Error (MAE): 1891.1708730866076
R-squared (R²): -0.28548002464340594


In [49]:
# Example manual input for XGBoost
manual_input = np.array([[2000, 2500, 2300]])  # Replace these values with INR.2 values of the last 3 months
manual_prediction_xgb = model.predict(manual_input)
print(f"XGBoost Prediction for next month: {manual_prediction_xgb[0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
XGBoost Prediction for next month: [1328.8193]


In [50]:
import numpy as np

# Example manual input for LSTM
manual_input_lstm = np.array([[2000.0, 2500.0, 2300.0]])  # Replace with the last 3 months' INR.2 values

# Ensure the shape is (samples, time steps, features)
manual_input_lstm = manual_input_lstm.reshape((manual_input_lstm.shape[0], manual_input_lstm.shape[1], 1))  # Reshape for LSTM

# Ensure the model is already trained
manual_prediction_lstm = model.predict(manual_input_lstm)

# Print the prediction
print(f"LSTM Prediction for next month: {manual_prediction_lstm[0][0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
LSTM Prediction for next month: 1328.8193359375
